In [77]:
import os
import sys
sys.path.append('utils')
from dataset.spot import DailyDataset, PeriodDataset, TOTAL_STD, TOTAL_MEAN, get_daily_spot, reduce_hour_mean, get_loader
from tool import to_gpu

import numpy as np
import pandas as pd
from pandas.tseries.offsets import Day


import matplotlib.pyplot as plt

import types

%matplotlib inline

In [16]:
path = './dataset'
TOTAL_STD = 22.732
TOTAL_MEAN = 38.99

In [14]:
day = pd.read_csv(os.path.join(path, 'day.csv'),
                   index_col='datetime', parse_dates=True)
day

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
datetime,,,,,,,,,,,,,,,,,,,,,
2011-12-19,0.181913,0.302396,0.249803,0.348691,0.365587,0.170174,-0.075734,0.033072,0.066110,-0.035824,...,0.126224,0.167463,0.156834,0.155083,0.295766,0.202879,0.204269,0.131771,0.115089,0.296152
2011-12-20,-0.035986,0.128604,0.133513,0.310839,0.286999,0.123465,-0.175842,-0.182480,-0.153933,-0.089047,...,-0.014818,0.070152,0.079023,0.148058,0.334183,0.385877,0.105734,-0.005767,0.034688,0.245209
2011-12-21,-0.071354,0.120019,0.149453,0.274511,0.259724,0.135284,-0.163736,-0.168765,-0.078811,-0.087497,...,-0.051381,0.047717,0.034872,0.027898,0.218763,0.112561,0.087373,-0.049770,0.020509,0.181657
2011-12-22,-0.225066,-0.030500,-0.033566,0.109074,0.073054,-0.064460,-0.308046,-0.400990,-0.349369,-0.381828,...,-0.187138,-0.039978,-0.022942,-0.017348,0.106706,-0.235454,-0.259782,-0.214428,-0.133055,-0.014162
2011-12-23,-0.327268,-0.140668,-0.179701,-0.074528,-0.107106,-0.157469,-0.390761,-0.400286,-0.305420,-0.386878,...,-0.250624,-0.139201,-0.211907,-0.169239,-0.045909,-0.465220,-0.376866,-0.278023,-0.195073,-0.094448
2011-12-24,-0.327749,-0.106050,-0.114527,0.003408,-0.038485,-0.146648,-0.433910,-0.500870,-0.392426,-0.458483,...,-0.301446,-0.139338,-0.160665,-0.175924,-0.028861,-0.262078,-0.260471,-0.177470,-0.183901,-0.120525
2011-12-25,-0.476646,-0.304683,-0.328810,-0.178170,-0.300469,-0.418161,-0.598270,-0.700502,-0.732530,-0.729710,...,-0.527661,-0.443860,-0.394865,-0.404344,-0.298211,-0.567773,-0.474865,-0.282234,-0.286951,-0.201929
2011-12-26,-0.422575,-0.280685,-0.337203,-0.208285,-0.272215,-0.319567,-0.567530,-0.617828,-0.571378,-0.654619,...,-0.329065,-0.268011,-0.180146,-0.254274,-0.201773,-0.503046,-0.508326,-0.380816,-0.221109,-0.134552
2011-12-27,-0.197544,-0.130735,-0.148077,-0.022647,-0.015186,-0.106483,-0.432564,-0.526872,-0.470363,-0.578342,...,-0.321972,-0.184929,-0.108430,-0.114468,0.031931,-0.232146,-0.467571,-0.267145,-0.144811,-0.040183


In [15]:
week = pd.read_csv(os.path.join(path, 'week.csv'),
                   index_col='datetime', parse_dates=True)
week

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
datetime,,,,,,,,,,,,,,,,,,,,,
2011-12-19,0.215659,0.006613,-0.091876,0.133539,-0.097635,-0.181059,-0.141183,-0.012558,-0.181801,-0.169404,...,0.157051,0.127751,0.321032,0.259220,0.416969,0.634472,0.415385,0.152389,0.049685,0.101854
2011-12-20,0.110962,0.034912,-0.006505,0.107509,-0.047614,-0.146148,-0.124522,-0.057832,-0.208251,-0.168432,...,0.046306,-0.004744,0.162995,0.118467,0.243694,0.389240,0.225332,0.067662,0.002943,0.022197
2011-12-21,0.024160,0.001363,-0.025163,0.046211,-0.079200,-0.134092,-0.078901,-0.047070,-0.246166,-0.214324,...,-0.013392,-0.082628,0.091212,0.031023,0.121835,0.161382,0.071285,0.001878,-0.003156,0.001498
2011-12-22,-0.020941,0.005451,0.007475,0.036478,-0.033590,-0.111982,-0.037545,-0.021961,-0.177599,-0.161136,...,-0.052113,-0.115560,0.029092,-0.018512,-0.003051,-0.026288,0.037573,-0.027513,-0.027373,-0.023606
2011-12-23,0.009259,0.042998,0.055038,0.070282,0.003399,-0.074607,-0.054282,-0.050988,-0.179644,-0.136930,...,-0.012634,-0.085923,0.041917,0.012131,0.047955,-0.012111,0.058544,0.009110,-0.014984,0.002262
2011-12-24,-0.035310,0.096281,0.073234,0.036925,-0.000124,-0.035496,-0.036751,-0.128081,-0.306813,-0.287085,...,-0.097894,-0.209653,-0.048082,-0.099652,-0.118369,-0.224067,-0.038008,-0.028824,-0.010872,0.004399
2011-12-25,-0.072136,0.009878,0.009872,-0.041434,0.017832,0.048672,-0.143644,-0.338270,-0.446481,-0.407504,...,-0.156943,-0.234007,-0.162520,-0.164144,-0.199087,-0.329889,-0.248386,-0.157375,-0.074306,-0.056635
2011-12-26,-0.099230,-0.084261,-0.073857,-0.045447,-0.009522,-0.069605,-0.035881,0.028818,-0.051004,-0.042946,...,0.046624,0.036741,0.111224,0.090214,0.123020,0.025214,0.076988,0.031246,0.010975,0.042273
2011-12-27,0.027632,0.029876,-0.029335,0.024478,-0.024166,-0.087511,-0.014128,0.018342,-0.130432,-0.128980,...,0.004460,-0.021847,0.093433,0.073177,0.056224,0.064796,0.145334,0.065601,-0.030143,0.042175


In [39]:
start_date = '2012-01-01'
end_date = '2016-06-30'
dt = pd.to_datetime(end_date)

In [40]:
t=week.loc[pd.to_datetime(start_date):pd.to_datetime(end_date)].values

In [41]:
t

array([[-0.11207492, -0.01224375,  0.02455396, ..., -0.3092294 ,
        -0.14997037, -0.14556114],
       [-0.10354   , -0.0922337 , -0.05790211, ..., -0.04224164,
        -0.03348773, -0.02534516],
       [-0.05742504, -0.08122887, -0.10049006, ..., -0.0150588 ,
        -0.0337239 , -0.0241482 ],
       ...,
       [-0.09038021, -0.0366924 , -0.00291857, ...,  0.03053609,
         0.03308648,  0.04456644],
       [-0.07676131, -0.01890462,  0.02687765, ..., -0.02057035,
         0.01543204,  0.03524301],
       [-0.06909911, -0.03846499,  0.02680144, ..., -0.12200463,
        -0.0390377 , -0.01389275]])

In [42]:
t.shape

(1643, 24)

In [43]:
spot = get_daily_spot(reduce_hour_mean())

In [44]:
s=spot.loc[pd.to_datetime(start_date):pd.to_datetime(end_date)].values

In [45]:
s

array([[-0.28434619, -0.37011616, -0.31080553, ..., -0.84520303,
        -0.90152894, -0.74140113],
       [-0.91187567, -0.98348058, -0.87485621, ..., -0.47273251,
        -0.57485289, -0.4925009 ],
       [-0.37109615, -0.38542497, -0.36772969, ..., -0.45922732,
        -0.65236476, -0.60133426],
       ...,
       [-0.48775988, -0.40161361, -0.29105364, ..., -0.48610573,
        -0.42273253, -0.60582133],
       [-0.61137417, -0.50103293, -0.44282207, ..., -0.46762957,
        -0.37434259, -0.53455615],
       [-0.56166451, -0.44824391, -0.37947525, ..., -0.63787416,
        -0.48695917, -0.63309566]])

In [46]:
s.shape

(1643, 24)

In [64]:
p = np.concatenate((t,s), axis=-1)

In [65]:
p

array([[-0.11207492, -0.01224375,  0.02455396, ..., -0.84520303,
        -0.90152894, -0.74140113],
       [-0.10354   , -0.0922337 , -0.05790211, ..., -0.47273251,
        -0.57485289, -0.4925009 ],
       [-0.05742504, -0.08122887, -0.10049006, ..., -0.45922732,
        -0.65236476, -0.60133426],
       ...,
       [-0.09038021, -0.0366924 , -0.00291857, ..., -0.48610573,
        -0.42273253, -0.60582133],
       [-0.07676131, -0.01890462,  0.02687765, ..., -0.46762957,
        -0.37434259, -0.53455615],
       [-0.06909911, -0.03846499,  0.02680144, ..., -0.63787416,
        -0.48695917, -0.63309566]])

In [66]:
p[:,0:24]=s

In [68]:
p[0]

array([-0.28434619, -0.37011616, -0.31080553, -0.36417902, -0.42572899,
       -0.62149535, -0.97991166, -1.73466673, -2.01395961, -2.12171181,
       -1.73718063, -1.61928819, -1.3799303 , -1.04771767, -1.09754181,
       -1.28449491, -1.23067661, -1.32781623, -1.08516463, -1.17127592,
       -0.93024364, -0.84520303, -0.90152894, -0.74140113, -0.28434619,
       -0.37011616, -0.31080553, -0.36417902, -0.42572899, -0.62149535,
       -0.97991166, -1.73466673, -2.01395961, -2.12171181, -1.73718063,
       -1.61928819, -1.3799303 , -1.04771767, -1.09754181, -1.28449491,
       -1.23067661, -1.32781623, -1.08516463, -1.17127592, -0.93024364,
       -0.84520303, -0.90152894, -0.74140113])

In [54]:
p=10+True+False

In [55]:
p

11

In [70]:
import torch
a=torch.Tensor([[1,2,3,4],[5,6,7,8],[9,10,11,12]])

In [71]:
a.shape

torch.Size([3, 4])

In [76]:
a[-2:-1,:]

tensor([[ 5.,  6.,  7.,  8.]])

In [74]:
for i in range(1, 1):
    print(i)

In [79]:
pd.to_datetime(start_date)<pd.to_datetime(end_date)

True

In [3]:
dataset = DailyDataset(N=2000, W=14)
loader = get_loader(dataset, batch_size=64, shuffle=True, num_workers=2)

Data build range: [window(2010-06-27 00:00:00) - 2010-07-11 00:00:00, 2015-12-31 00:00:00]


In [4]:
trainX, trainY = dataset.get_io('2012-01-01', '2015-12-31')
testX, testY = dataset.get_io('2012-01-01', '2016-08-30')

In [6]:
trainX.shape

torch.Size([1474, 24])

In [7]:
trainY.shape

torch.Size([1461, 24])

In [8]:
testX.shape

torch.Size([1717, 24])

In [9]:
testY.shape

torch.Size([1704, 24])

In [11]:
enumerate(loader)

In [2]:
spot=get_daily_spot(reduce_hour_mean())

In [34]:
end_date='2015-12-31'
dt = pd.to_datetime(end_date)

In [35]:
sliding = spot.loc[pd.to_datetime(dt) - Day(14+1):pd.to_datetime(dt)].values

In [36]:
sliding

array([[-5.97289152e-02, -1.19632258e-01, -6.71402142e-02,
         4.71354279e-02,  1.04492721e-01,  5.66675916e-02,
        -1.62165749e-01,  1.37935854e-02, -2.74194576e-03,
        -1.21579839e-01, -2.13337586e-01, -2.90764523e-01,
        -1.40444115e-01, -1.11460408e-01,  2.75681714e-02,
         6.53780509e-04,  1.13287850e-01,  5.16048500e-02,
        -4.30653860e-02, -4.36100837e-01, -4.06224636e-01,
        -2.78908823e-01, -3.86220121e-01, -3.66071197e-01],
       [ 1.41749177e-01, -4.58801711e-01, -3.64518360e-01,
        -2.66519332e-01, -2.25438653e-01, -2.57427077e-01,
        -3.65843384e-01, -3.61448364e-01, -2.26215463e-01,
        -2.99302873e-01, -3.45750044e-01, -3.97222380e-01,
        -2.78135476e-01, -3.80244501e-01, -3.09841647e-01,
        -2.39536260e-01, -1.58135694e-01, -5.74924577e-02,
        -2.22108145e-01, -5.77751373e-01, -5.43476088e-01,
        -3.66450615e-01, -5.85498671e-01, -3.43195955e-01],
       [-9.40417781e-02, -2.09373592e-01, -1.56441639e

In [42]:
p=sliding[13:15].reshape(24*2)
p

array([-0.53527   , -0.55030268, -0.55675837, -0.45040108, -0.46694842,
       -0.51037446, -0.57304029, -0.53873149, -0.78094008, -0.78408204,
       -0.44472946, -0.46188893, -0.22842582, -0.10486178, -0.12420026,
       -0.20742294, -0.1220632 ,  0.05776357,  0.24419486,  0.1195036 ,
       -0.06617537, -0.31762077, -0.37390268, -0.38850653, -0.35138825,
       -0.18781808, -0.09617418, -0.16534038, -0.29318456, -0.44614782,
       -0.48329895, -0.69137974, -0.72463179, -0.49638188, -0.57318274,
       -0.50060088, -0.53636176, -0.55972717, -0.61249869, -0.56066946,
       -0.49554551, -0.43185459, -0.31624856, -0.6309803 , -0.55227426,
       -0.38316714, -0.74914463, -0.3607923 ])

In [46]:
a=[]
a += [p]
a += [p]

In [47]:
a

[array([-0.53527   , -0.55030268, -0.55675837, -0.45040108, -0.46694842,
        -0.51037446, -0.57304029, -0.53873149, -0.78094008, -0.78408204,
        -0.44472946, -0.46188893, -0.22842582, -0.10486178, -0.12420026,
        -0.20742294, -0.1220632 ,  0.05776357,  0.24419486,  0.1195036 ,
        -0.06617537, -0.31762077, -0.37390268, -0.38850653, -0.35138825,
        -0.18781808, -0.09617418, -0.16534038, -0.29318456, -0.44614782,
        -0.48329895, -0.69137974, -0.72463179, -0.49638188, -0.57318274,
        -0.50060088, -0.53636176, -0.55972717, -0.61249869, -0.56066946,
        -0.49554551, -0.43185459, -0.31624856, -0.6309803 , -0.55227426,
        -0.38316714, -0.74914463, -0.3607923 ]),
 array([-0.53527   , -0.55030268, -0.55675837, -0.45040108, -0.46694842,
        -0.51037446, -0.57304029, -0.53873149, -0.78094008, -0.78408204,
        -0.44472946, -0.46188893, -0.22842582, -0.10486178, -0.12420026,
        -0.20742294, -0.1220632 ,  0.05776357,  0.24419486,  0.1195036 ,
  

In [49]:
b=np.array(a)

In [50]:
b.shape

(2, 48)

In [ ]:
tmp=[]
idx=0
for i in range(14):
    tmp += [sliding[i:i+2].reshape(24*2)]


In [20]:
def _sliding_window(date):
    _sliding = spot.loc[pd.to_datetime(date) - Day(15):
                             pd.to_datetime(date)].values
    x = _sliding[:15]
    y = _sliding[-14:]
    return x, y

In [26]:
end_date='2015-12-31'
dt = pd.to_datetime(end_date)
X_ = []
Y_ = []
dates_ = []
for _ in range(2):
    x, y = _sliding_window(dt)
    X_.append(x)
    Y_.append(y)
    dates_.append(dt)
    dt -= Day(1)

In [13]:
sliding.shape

(16, 24)

In [27]:
X_

[array([[-5.97289152e-02, -1.19632258e-01, -6.71402142e-02,
          4.71354279e-02,  1.04492721e-01,  5.66675916e-02,
         -1.62165749e-01,  1.37935854e-02, -2.74194576e-03,
         -1.21579839e-01, -2.13337586e-01, -2.90764523e-01,
         -1.40444115e-01, -1.11460408e-01,  2.75681714e-02,
          6.53780509e-04,  1.13287850e-01,  5.16048500e-02,
         -4.30653860e-02, -4.36100837e-01, -4.06224636e-01,
         -2.78908823e-01, -3.86220121e-01, -3.66071197e-01],
        [ 1.41749177e-01, -4.58801711e-01, -3.64518360e-01,
         -2.66519332e-01, -2.25438653e-01, -2.57427077e-01,
         -3.65843384e-01, -3.61448364e-01, -2.26215463e-01,
         -2.99302873e-01, -3.45750044e-01, -3.97222380e-01,
         -2.78135476e-01, -3.80244501e-01, -3.09841647e-01,
         -2.39536260e-01, -1.58135694e-01, -5.74924577e-02,
         -2.22108145e-01, -5.77751373e-01, -5.43476088e-01,
         -3.66450615e-01, -5.85498671e-01, -3.43195955e-01],
        [-9.40417781e-02, -2.09373592e

In [14]:
x = sliding[:15]
y = sliding[-14:]

In [28]:
Y_

[array([[-0.09404178, -0.20937359, -0.15644164, -0.06372151, -0.12293997,
         -0.15448849, -0.1700841 , -0.13885466, -0.11843788, -0.3265772 ,
         -0.39765925, -0.38930403, -0.40394931, -0.19416321, -0.18270809,
         -0.23029818, -0.18277057, -0.06453099, -0.19571364, -0.48537059,
         -0.39830628, -0.44475433, -0.57758032, -0.48088732],
        [ 0.03133214, -0.18825798, -0.1907545 , -0.176778  , -0.10974272,
         -0.2847014 , -0.56160267, -0.77100318, -0.83856809, -0.77528387,
         -0.56394466, -0.59298166, -0.55923701, -0.45414913, -0.50956011,
         -0.46696895, -0.33365918, -0.32935591, -0.31140957, -0.63273994,
         -0.85097213, -0.73905311, -0.82964789, -0.58470572],
        [-0.46576446, -0.58329582, -0.64869925, -0.57269565, -0.58572371,
         -0.59747635, -1.05649973, -1.30461219, -1.51734691, -1.40875211,
         -1.39097264, -1.34786465, -1.20634241, -1.10037471, -0.92263419,
         -0.93151233, -0.93061502, -0.70063868, -0.69412996, -

In [15]:
x

array([[-5.97289152e-02, -1.19632258e-01, -6.71402142e-02,
         4.71354279e-02,  1.04492721e-01,  5.66675916e-02,
        -1.62165749e-01,  1.37935854e-02, -2.74194576e-03,
        -1.21579839e-01, -2.13337586e-01, -2.90764523e-01,
        -1.40444115e-01, -1.11460408e-01,  2.75681714e-02,
         6.53780509e-04,  1.13287850e-01,  5.16048500e-02,
        -4.30653860e-02, -4.36100837e-01, -4.06224636e-01,
        -2.78908823e-01, -3.86220121e-01, -3.66071197e-01],
       [ 1.41749177e-01, -4.58801711e-01, -3.64518360e-01,
        -2.66519332e-01, -2.25438653e-01, -2.57427077e-01,
        -3.65843384e-01, -3.61448364e-01, -2.26215463e-01,
        -2.99302873e-01, -3.45750044e-01, -3.97222380e-01,
        -2.78135476e-01, -3.80244501e-01, -3.09841647e-01,
        -2.39536260e-01, -1.58135694e-01, -5.74924577e-02,
        -2.22108145e-01, -5.77751373e-01, -5.43476088e-01,
        -3.66450615e-01, -5.85498671e-01, -3.43195955e-01],
       [-9.40417781e-02, -2.09373592e-01, -1.56441639e

In [16]:
y

array([[-0.09404178, -0.20937359, -0.15644164, -0.06372151, -0.12293997,
        -0.15448849, -0.1700841 , -0.13885466, -0.11843788, -0.3265772 ,
        -0.39765925, -0.38930403, -0.40394931, -0.19416321, -0.18270809,
        -0.23029818, -0.18277057, -0.06453099, -0.19571364, -0.48537059,
        -0.39830628, -0.44475433, -0.57758032, -0.48088732],
       [ 0.03133214, -0.18825798, -0.1907545 , -0.176778  , -0.10974272,
        -0.2847014 , -0.56160267, -0.77100318, -0.83856809, -0.77528387,
        -0.56394466, -0.59298166, -0.55923701, -0.45414913, -0.50956011,
        -0.46696895, -0.33365918, -0.32935591, -0.31140957, -0.63273994,
        -0.85097213, -0.73905311, -0.82964789, -0.58470572],
       [-0.46576446, -0.58329582, -0.64869925, -0.57269565, -0.58572371,
        -0.59747635, -1.05649973, -1.30461219, -1.51734691, -1.40875211,
        -1.39097264, -1.34786465, -1.20634241, -1.10037471, -0.92263419,
        -0.93151233, -0.93061502, -0.70063868, -0.69412996, -0.81882123,
  

In [29]:
start_date = dates[-1]

for _ in range(15):
    dates_.append(dt)
    dt -= Day(1)

X = np.array(list(reversed(X_)))
Y = np.array(list(reversed(Y_)))
dates = np.array(list(reversed(dates_)))

In [31]:
X.shape

(2, 15, 24)

In [32]:
Y.shape

(2, 14, 24)

In [33]:
dates

array([Timestamp('2015-12-15 00:00:00'), Timestamp('2015-12-16 00:00:00'),
       Timestamp('2015-12-17 00:00:00'), Timestamp('2015-12-18 00:00:00'),
       Timestamp('2015-12-19 00:00:00'), Timestamp('2015-12-20 00:00:00'),
       Timestamp('2015-12-21 00:00:00'), Timestamp('2015-12-22 00:00:00'),
       Timestamp('2015-12-23 00:00:00'), Timestamp('2015-12-24 00:00:00'),
       Timestamp('2015-12-25 00:00:00'), Timestamp('2015-12-26 00:00:00'),
       Timestamp('2015-12-27 00:00:00'), Timestamp('2015-12-28 00:00:00'),
       Timestamp('2015-12-29 00:00:00'), Timestamp('2015-12-30 00:00:00'),
       Timestamp('2015-12-31 00:00:00')], dtype=object)

In [ ]:
def get_io(self, start_date, end_date):
    _sliding = self.spot.loc[pd.to_datetime(start_date) - Day(self.W):
                             pd.to_datetime(end_date)].values
    i_stream = _sliding[:-1]
    o_stream = _sliding[1:]
    return torch.from_numpy(i_stream).float(), torch.from_numpy(o_stream).float()

In [6]:
dataset = PeriodDataset(N=2000, W=14, P=7)
loader = get_loader(dataset, batch_size=64, shuffle=True, num_workers=2)

Data build range: [window(2010-06-21 00:00:00) - 2010-07-11 00:00:00, 2015-12-31 00:00:00]


In [7]:
trainX, trainY = dataset.get_io('2012-01-01', '2015-12-31')
testX, testY = dataset.get_io('2012-01-01', '2016-08-30')

(1481, 24)
(1724, 24)


In [8]:
trainX.shape

torch.Size([1473, 168])

In [9]:
trainY.shape

torch.Size([1473, 24])